
# OpenProBono RAG Evaluation

This notebook is based on [RAG Evaluation](https://huggingface.co/learn/cookbook/en/rag_evaluation) by [Aymeric Roucher](https://huggingface.co/m-ric).

Any section inside block quotes is a direct quote. The general structure is copied and ideas are paraphrased. Prompts are adjusted to fit OpenProBono's use case. Code has been added and modified.

>This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.
>
>For an introduction to RAG, you can check [this other cookbook](https://huggingface.co/learn/cookbook/en/rag_zephyr_langchain)!
>
>RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:
>
><img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" alt="RAG workflow: Knowledge Base, Embedding Model, LLM, LLM Prompt, and more" height="700"/>
>
>Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system’s performance! So let’s see how to evaluate our RAG system.
>

### Evaluating RAG performance

>Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.
>
>For our evaluation pipeline, we will need:
>
>1. An evaluation dataset with question - answer couples (QA couples)
>2. An evaluator to compute the accuracy of our system on the above evaluation dataset.
>
>➡️ It turns out, we can use LLMs to help us all along the way!
>
>1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
>2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.
>
>**Let’s dig into it and start building our evaluation pipeline!**

### 0: Install and import dependencies

In [1]:
%pip install -q tqdm openai pandas langchain unstructured pymilvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from pathlib import Path

import openai
import pandas as pd
import pymilvus
from tqdm.auto import tqdm

import encoders
import milvusdb
from models import EncoderParams, OpenAIModelEnum

pd.set_option("display.max_colwidth", None)

/Users/njc/Documents/programming/opb/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1: Build a synthetic dataset for evaluation

>We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.
>
>Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

#### 1.1: Load sources
 
We have a list of sources we use to generate questions about one source at a time using a generator function. The sources can be files or URLs.

Our current sources are the NC General Statutes. We can access them by chapter or by statute.

In [14]:
from unstructured.partition.auto import partition

# for loading by statute
root_dir = "data/chapter_urls/"
chapter_names = sorted(os.listdir(root_dir))
# for loading by chapter
with Path("data/NC/urls-employment").open() as f:
    chapter_pdf_urls = [line.strip() for line in f.readlines()]

def load_statute_urls(chapter_name: str):
    with Path(root_dir + chapter_name).open() as f:
        return [line.strip() for line in f.readlines()]

def generate_statute_elements():
    for chapter in chapter_names:
        statute_urls = load_statute_urls(chapter)
        for statute_url in statute_urls:
            elements = partition(statute_url)
            yield statute_url, elements

def resume_statute_elements(chapter_name: str, statute_url: str):
    resume_chapter = next(iter([chapter for chapter in chapter_names if chapter == chapter_name]), None)
    resume_chapter_idx = chapter_names.index(resume_chapter) if resume_chapter else 0
    for i in range(resume_chapter_idx, len(chapter_names)):
        statute_urls = load_statute_urls(chapter_names[i])
        resume_statute = next(iter([statute for statute in statute_urls if statute == statute_url]), None)
        resume_statute_idx = statute_urls.index(resume_statute) if resume_statute else 0
        for j in range(resume_statute_idx, len(statute_urls)):
            elements = partition(url=statute_urls[j], content_type="application/pdf")
            yield statute_url, elements

def generate_chapter_elements():
    for chapter_pdf_url in chapter_pdf_urls:
        elements = partition(url=chapter_pdf_url, content_type="application/pdf")
        yield chapter_pdf_url, elements

We prepare the sources for question generation by *chunking* them. We will do this again later using different chunking strategies when we store embedded chunks in a vector database for RAG.

In [12]:
from unstructured.chunking.title import chunk_by_title
from unstructured.documents.elements import Element


def chunk_elements_qa(elements: list[Element]) -> list[Element]:
    return chunk_by_title(
        elements,
        max_characters=10000,
        combine_text_under_n_chars=500,
        new_after_n_chars=2500,
        overlap=1000,
    )

#### 1.2 Setup question generation agents

It is necessary to call an LLM for each agent in the eval dataset generation and RAG processes, so we use our internal chat function. We'll use `gpt-3.5-turbo` for the model.

In [54]:
from chat_models import chat, messages
from models import ChatModelParams

cm_params = ChatModelParams(engine="openai", model="gpt-4o")

Here is the prompt that is given to our question generation agent:

In [55]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

For cost and time considerations, we generate a number of questions proportional to the length of the chunk.

In [56]:
import random


def generate_qa_couples(
    chunks: list[Element],
    max_num_questions: int,
    min_num_chars: int,
    chars_per_page: int,
    max_len_answer: int,
) -> list:
    # filter out chunks with length < min_num_chars
    chunks = [chunk for chunk in chunks if len(chunk.text) >= min_num_chars]
    char_count = sum([len(chunk.text) for chunk in chunks])
    if char_count < min_num_chars:
        return []

    question_count = min([max_num_questions, max([1, char_count // chars_per_page])])
    print(f"Generating {question_count} QA couples...")

    couples = []
    for sampled_context in tqdm(random.sample(chunks, question_count)):
        # Generate QA couple
        response = chat(
            messages(
                [[QA_generation_prompt.format(context=sampled_context.text), None]],
                cm_params.engine,
            ),
            cm_params,
            temperature=0.7,
        )
        output_qa_couple = response.choices[0].message.content
        try:
            question = output_qa_couple.split("Factoid question: ")[-1].split("Answer: ")[0].rstrip()
            answer = output_qa_couple.split("Answer: ")[-1].rstrip()
            assert len(answer) < max_len_answer, "Answer is too long"
            couples.append(
                {
                    "context": sampled_context.text,
                    "question": question,
                    "answer": answer,
                    "source_doc": sampled_context.metadata.url,
                },
            )
        except Exception as e:
            print(e)
    return couples

In [57]:
def generate_n_qa_couples(
    n: int,
    max_questions_per_chunk: int = 4,
    min_chars_per_chunk: int = 500,
    chars_per_page: int = 2500,
    max_len_answer: int = 1000,
):
    if n < max_questions_per_chunk:
        # so we generate exactly n couples
        max_questions_per_chunk = n
    tot_couples = []
    for src, elements in generate_chapter_elements():
        chunks = chunk_elements_qa(elements)
        couples = generate_qa_couples(
            chunks,
            # so we generate exactly n couples
            min([max_questions_per_chunk, n - len(tot_couples)]),
            min_chars_per_chunk,
            chars_per_page,
            max_len_answer,
        )
        if couples:
            print(couples[0]["source_doc"])
            for couple in couples:
                print(couple["question"])
                print(couple["answer"])
                print()
        tot_couples += couples
        if len(tot_couples) == n:
            break
    return tot_couples

In [58]:
NUM_QUESTIONS = 100
couples = generate_n_qa_couples(NUM_QUESTIONS)
display(pd.DataFrame(couples).head(NUM_QUESTIONS))

Generating 4 QA couples...


100%|██████████| 4/4 [00:05<00:00,  1.36s/it]


https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_83A.pdf
Who can file a charge of unprofessional conduct with the Board?
Any person

What is the maximum total value for an institutional or commercial building to be exempt from requiring an architectural license?
Three hundred thousand dollars ($300,000)

Who can initiate actions to recover civil penalties?
The Attorney General or any private counsel retained under G.S. 114-2.3.

What is the exemption value threshold for a commercial building project to not require a professional architectural seal?
Two hundred thousand dollars ($200,000)

Generating 4 QA couples...


100%|██████████| 4/4 [00:05<00:00,  1.32s/it]


https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_84.pdf
What must a corporation comply with when performing acts in connection with its fiduciary activities?
The corporation shall comply with the specified regulations.

Who has the authority to enter orders necessary to protect the interests of clients of missing, suspended, disbarred, disabled, or deceased attorneys in North Carolina?
The senior regular resident judge of the superior court of any district wherein a member of the North Carolina State Bar resides or maintains an office.

Which Session Laws repealed certain regulations in 1991?
Session Laws 1991, c. 210, s. 1

Who can the Council appoint to represent the North Carolina State Bar in proceedings?
A member of the North Carolina State Bar.

Generating 4 QA couples...


...

100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_93D.pdf
When were sections (15) and (16) repealed by Session Laws?
August 11, 2014

What is the North Carolina State Hearing Aid Dealers and Fitters Board responsible for?
The North Carolina State Hearing Aid Dealers and Fitters Board is responsible for licensing hearing aid specialists.

What is the maximum fee for obtaining a license certificate after passing the examination?
Twenty-five dollars ($25.00)

What is the maximum fee set by the Board for reissuing a suspended license?
Two hundred dollars ($200.00).

Generating 2 QA couples...


100%|██████████| 2/2 [00:02<00:00,  1.18s/it]

https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_93E.pdf
How much is the nonrefundable fee for the approval of a qualifying course for private real estate appraisal schools?
One hundred dollars ($100.00) per course.

What may constitute grounds for the Board to deny registration to an applicant or registrant?
Refusal to consent to a criminal history record check.



,context,question,answer,source_doc
0,"§ 83A-14. Disciplinary action and procedure.\n\nAny person may file with the Board a charge of unprofessional conduct, negligence, incompetence, dishonest practice, or other misconduct or of any violation of this Chapter or of a Board rule adopted and published by the Board. Upon receipt of such charge, or upon its own initiative, the Board may give notice of an administrative hearing under the Administrative Procedure Act, or may dismiss the charge as unfounded or trivial, upon a statement of the reasons therefor which shall be mailed to the architect or registered interior designer and the person who filed the charge by registered or certified mail. (1979, c. 871, s. 1; 2021-81, s. 1.)",Who can file a charge of unprofessional conduct with the Board?,Any person,https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_83A.pdf
1,"§ 83A-13. Exemptions.\n\nNothing in this Chapter shall be construed to prevent the practice of general contracting under the provisions of Article 1 of Chapter 87, or the practice by any person who is qualified under law as a ""registered professional engineer"" of such architectural work as is incidental to engineering projects or utilities, or the practice of any other profession under the applicable licensure provisions of the General Statutes.\n\n(a)\n\nNothing in this Chapter shall be construed to prevent a duly licensed general contractor, professional engineer, architect, or registered interior designer acting individually or in combination thereof, from participating in a ""Design/Build"" undertaking including the preparation of plans and/or specifications and entering individual or collective agreements with the owner in order to meet the owner's requirements for pre-determined costs and unified control in the design and construction of a project, and for the method of compensation for the design and construction services rendered; provided, however, that nothing herein shall be construed so as to allow the performance of any such services or any division thereof by one who is not duly licensed to perform such service or services in accordance with applicable licensure provisions of the General Statutes; provided further, that full disclosure is made in writing to the owner as to the duties and responsibilities of each of the participating parties in such agreements; and, provided further, nothing in this Chapter shall prevent the administration by any of the said licensees of construction contracts and related services or combination of services in connection with the construction of buildings. (c)\n\n(b)\n\n(Effective until December 31, 2024 – see note) Nothing in this Chapter shall be construed to require an architectural license for the preparation, sale, or furnishing of plans, specifications and related data, or for the supervision of construction pursuant thereto, where the building, buildings, or project involved is in one of the following categories:\n\n(1)\n\n(2)\n\n(3)\n\n(4)\n\n(5)\n\nA family residence, up to eight units attached with grade level exit, which is not a part of or physically connected with any other buildings or residential units; A building upon any farm for the use of any farmer, unless the building is of such nature and intended for such use as to substantially involve the health or safety of the public; An institutional or commercial building if it does not have a total value exceeding three hundred thousand dollars ($300,000); An institutional or commercial building if the total building area does not exceed 3,000 square feet in gross floor area; Alteration, remodeling, or renovation of an existing building that is exempt under this section, or alteration, remodeling, or renovation of an existing building or building site that does not alter or affect the structural system of the building; change the building's access or exit pattern; or change the live or dead load on the building's structural system. This s

#### 1.5 Setup question critique agents

The generated questions can be flawed in many ways.

>We use an agent to determine if a generated question meets the following criteria, given in [this paper](https://huggingface.co/papers/2312.10003):

- **Groundedness**: can the question be answered from the given context?
- **Relevance**: is the question relevant to users? For instance, *"What are some of Thomas Jefferson's beliefs regarding the rights and liberties of individuals?"* is not relevant for OpenProBono users.

>One last failure case we’ve noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like *"What is the name of the function used in this guide?"*. We also build a critique agent for this criteria:

- **Standalone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? For instance, *"What does the term 'legal entity' refer to in this statute?"* is tailored for a particular statute, but unclear by itself.

>We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.
>
>💡 ***When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.***"
>
>We now build and run these critique agents.

In [59]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to people learning about the legal system.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question cannot be understood at all without the context, and 5 means that the entire question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'according to this Article', the rating must be 1.
The questions can contain obscure legal definitions or entities like trier of fact or the North Carolina Self-Insurance Security Fund and still be a 5: it must simply be clear to an operator with access to legal documents what the question is about.

For instance, the context "On July 1 of each year, a maximum weekly benefit amount shall be computed." and question "When is the maximum weekly benefit amount computed and adjusted?" should receive a 1, since the question implicitly mentions the maximum weekly benefit amount, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the context and question.

Context: {context}\n
Question: {question}\n
Answer::: """

In [60]:
critique_cm = ChatModelParams(engine="openai", model="gpt-4o")

def generate_qa_critiques(couples: list[dict]):
    print("Generating critique for each QA couple...")
    for output in tqdm(couples):
        evaluations = {
            "groundedness": chat(
                messages(
                    [[question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]), None]],
                    critique_cm.engine,
                ),
                critique_cm,
            ).choices[0].message.content,
            "relevance": chat(
                messages(
                    [[question_relevance_critique_prompt.format(question=output["question"]), None]],
                    critique_cm.engine,
                ),
                critique_cm,
            ).choices[0].message.content,
            "standalone": chat(
                messages(
                    [[question_standalone_critique_prompt.format(context=output["context"], question=output["question"]), None]],
                    critique_cm.engine,
                ),
                critique_cm,
            ).choices[0].message.content,
        }
        try:
            for criterion, evaluation in evaluations.items():
                score, eval = (
                    int(evaluation.split("Total rating: ")[-1].strip()),
                    evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
                )
                output.update(
                    {
                        f"{criterion}_score": score,
                        f"{criterion}_eval": eval,
                    }
                )
        except Exception as e:
            print(e)
            continue

>Now let us filter out bad questions based on our critique agent scores:

In [61]:
def filter_questions(
        generated_questions: pd.DataFrame,
        min_groundedness: int,
        min_relevance: int,
        min_standalone: int,
):
    return generated_questions.loc[
        (generated_questions["groundedness_score"] >= min_groundedness)
        & (generated_questions["relevance_score"] >= min_relevance)
        & (generated_questions["standalone_score"] >= min_standalone)
    ]

In [64]:
#generate_qa_critiques(couples)
couples_df = pd.DataFrame.from_dict(couples)
with Path("./employment_dataset-critiques.json").open("w") as f:
    f.write(couples_df.to_json())
#display(couples_df.head(NUM_QUESTIONS))

In [65]:
couples_df = filter_questions(couples_df, 4, 4, 4)
display(couples_df.head(NUM_QUESTIONS))

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
0,"§ 83A-14. Disciplinary action and procedure.\n\nAny person may file with the Board a charge of unprofessional conduct, negligence, incompetence, dishonest practice, or other misconduct or of any violation of this Chapter or of a Board rule adopted and published by the Board. Upon receipt of such charge, or upon its own initiative, the Board may give notice of an administrative hearing under the Administrative Procedure Act, or may dismiss the charge as unfounded or trivial, upon a statement of the reasons therefor which shall be mailed to the architect or registered interior designer and the person who filed the charge by registered or certified mail. (1979, c. 871, s. 1; 2021-81, s. 1.)",Who can file a charge of unprofessional conduct with the Board?,Any person,https://www.ncleg.gov/EnactedLegislation/Statutes/PDF/ByChapter/Chapter_83A.pdf,5,"The context clearly states that ""Any person may file with the Board a charge of unprofessional conduct."" This directly answers the question of who can file a charge of unprofessional conduct with the Board. The information is explicit and leaves no room for ambiguity.\n\n",4,"This question is quite useful for individuals learning about the legal system, particularly in the context of professional regulation and accountability. Understanding who has the authority to file a charge of unprofessional conduct with a regulatory board is crucial for both professionals and the public. It helps in comprehending the mechanisms of oversight and the processes involved in maintaining professional standards.\n\n",5,"The question ""Who can file a charge of unprofessional conduct with the Board?"" is clear and understandable without needing the specific context provided. It asks about the general ability of individuals to file charges of unprofessional conduct with a governing board, which is a concept that can be understood independently of the specific details in the context.\n\n"
2,"factors:\n\nThe nature, gravity, and persistence of the particular violation. The appropriateness of the imposition of a civil penalty when considered alone or in combination with other punishment. (3)\t Whether the violation was willful and malicious. (4)\n\n(1)\t (2)\n\nAny other factors that would tend to mitigate or aggravate the violations found to exist.\n\nActions and prosecutions under this section shall be commenced in the county in which the defendant resides, or has his principal place of business, or in the case of an out-of-state corporation, is conducting business.\n\n(b)\n\n(c)\n\nActions to recover civil penalties shall be initiated by the Attorney General, or any\n\nprivate counsel retained under G.S. 114-2.3.\n\n(d)\n\nThe Board shall establish a schedule of civil penalties for violations of this Chapter and\n\nrules adopted by the Board.\n\nThe Board may in a disciplinary proceeding charge costs, including reasonable attorneys' fees, to the licensee or registered interior designer against whom the proceedings were brought. (1915, c. 270, s. 4; C.S., s. 4996; 1941, c. 369, ss. 1, 2; 1951, c. 1130, s. 3; 1957, c. 794, s. 11; 1965, c. 1100; 1969, c. 718, s. 21; 1973, c. 1414, s. 1; 1979, c. 871, s. 1; 1993, c. 539, s. 595; 1994, Ex. Sess., c. 24, s. 14(c); 1998-215, s. 129; 2021-81, s. 1.)\n\n(e)\n\n§ 83A-17. Power of Board to seek injunction.\n\nThe Board may appear in its own name and apply to courts having jurisdiction for injunctions to prevent violations of this Chapter or of rules issued pursuant thereto, and such courts are empowered to grant such injunctions regardless of whether criminal prosecution or other action has been or may be instituted as a result of such violation. A single act of unauthorized or illegal practice shall be sufficient, if shown, to invoke the injunctive relief of this section or criminal penalties under G.S. 83A-16. (1979, c. 871, s.

>Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

Save the dataset to a file:

In [66]:
with Path("./employment_dataset-filtered.json").open("w") as f:
    f.write(couples_df.to_json())

Read the dataset from the file:

In [ ]:
couples_df = pd.read_json("./employment_dataset.json")
display(couples_df.head(NUM_QUESTIONS))

### 2: Build our RAG System

We load the data for RAG into a `Collection` in our Milvus vector database.

To accomplish this, we must:

1. extract the content from our sources
2. chunk the extracted content
3. embed the chunks
4. insert them to the database

We already wrote generator functions to extract the content, `generate_statute_elements` and `generate_chapter_elements`, so our next step is chunking.

#### 2.1: Chunk sources into documents

>- In this part, **we split the documents from our knowledge base into smaller chunks**: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
>- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.
>
>Many options exist for text splitting:
>
>- split every n words / characters, but this has the risk of cutting in half paragraphs or even sentences
>- split after n words / character, but only on sentence boundaries
>- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).
>
>To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.
>
>[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

We need a function to chunk the content in a given source into documents. This function should accept a list of items containing extracted text and relevant metadata from the source. In our generator function, `partition` returns a list of `Element` objects. We pass these into a chunking function, `chunk_by_title`.

In [ ]:
from unstructured.chunking.title import chunk_by_title

chunking_function = chunk_by_title

#### 2.2: Embed documents

>The retriever acts like an internal search engine: given the user query, it returns the most relevant documents from your knowledge base.

An embedding model transforms the documents to vectors, and Milvus creates an index over those vectors for fast and accurate retrieval.

We want to evaluate different embedding models, so we represent them with a function that accepts text and returns vectors. In order to easily test different models, the function also accepts an `EncoderParams` object that tells it which embedding model to use.

In [ ]:
embedding_function = encoders.embed_strs

#### 2.3: Create vector database

The document embeddings are stored in a vector database for fast vector searching during RAG.

Below is a function that creates a vector database we can use to store and retrieve the embedded documents. The `name` of the database will be determined by the chosen sources, chunking strategy, and embedding model evaluation parameters.

In [ ]:
create_collection_function = milvusdb.create_collection

#### 2.4: Insert documents

The last step is to insert the embedded documents into the newly created vector database.

An embedded document is represented by its corresponding `vectors`, `texts`, and `metadatas`. The vector database is given by a Milvus `Collection`.

In [ ]:
INSERT_BATCH_SIZE = 100

def insert_documents(vectors: list, texts: list, metadatas: list, collection: pymilvus.Collection):
    data = [vectors, texts, metadatas]
    num_chunks = len(vectors)
    pks = []
    for i in range(0, num_chunks, INSERT_BATCH_SIZE):
        batch_vector = data[0][i: i + INSERT_BATCH_SIZE]
        batch_text = data[1][i: i + INSERT_BATCH_SIZE]
        batch_metadata = data[2][i: i + INSERT_BATCH_SIZE]
        batch = [batch_vector, batch_text, batch_metadata]
        current_batch_size = len(batch[0])
        res = collection.insert(batch)
        pks += res.primary_keys
        if res.insert_count != current_batch_size:
            # the upload failed, try deleting any partially uploaded data
            bad_deletes = []
            for pk in pks:
                delete_res = collection.delete(expr=f"pk=={pk}")
                if delete_res.delete_count != 1:
                    bad_deletes.append(pk)
            bad_insert = (
                f"Failure: expected {current_batch_size} insertions but got "
                f"{res.insert_count}. "
            )
            if bad_deletes:
                bad_insert += f"Dangling data: {','.join(bad_deletes)}"
            else:
                bad_insert += "Any partially uploaded data has been deleted."
            return {"message": bad_insert}
    return {"message": "Success", "num_chunks": num_chunks}

#### 2.5 Populate the vector database

So far, the parameters we can control are:

1. Chunking strategy parameters
    - chunk hardmax: the maximum number of characters in a chunk
    - chunk softmax: the preferred maximum number of characters in a chunk (see `new_after_n_chars` in `chunk_by_title` for more)
    - overlap: the number of characters to overlap between consecutive chunks
2. Embedding model

There are also some parameters we can't currently control:

1. Document loader (unstructured)
2. Question generation chunking strategy (LangChain's `RecursiveCharacterTextSplitter`)
2. Base RAG chunking strategy (unstructured's `chunk_by_title`)
3. Vector Index (Zilliz autoindex)

Once we decide on values for the parameters we can control, we can chunk and embed our sources into a vector database. We can then use the database to generate a synthetic dataset of question-answer pairs and/or evaluate agents' responses to a generated (or loaded) dataset of questions.

In [ ]:
def populate_database(collection: pymilvus.Collection, chunk_hardmax: int, chunk_softmax: int, overlap: int, encoder: EncoderParams):
    for src, elements in generate_chapter_elements():
        chunks = chunking_function(
            elements,
            max_characters=chunk_hardmax,
            new_after_n_chars=chunk_softmax,
            overlap=overlap,
        )
        num_chunks = len(chunks)
        texts, metadatas = [], []
        for i in range(num_chunks):
            texts.append(chunks[i].text)
            metadatas.append(chunks[i].metadata.to_dict())
        vectors = embedding_function(texts, encoder)
        result = insert_documents(vectors, texts, metadatas, collection)
        if result["message"] != "Success":
            break

We may want to rerun this notebook with data that has already been inserted to Milvus. If so, we need different generator functions to load the data from Milvus rather than the source URLs themselves.

In [ ]:
from unstructured.documents.elements import ElementMetadata, Text


def vdb_source_documents(collection_name: str, source: str):
    expr = f"metadata['url']=='{source}'"
    hits = milvusdb.get_expr(collection_name, expr)["result"]
    for i in range(len(hits)):
        hits[i]["url"] = hits[i]["metadata"]["url"]
        hits[i]["page_number"] = hits[i]["metadata"]["page_number"]
        del hits[i]["pk"]
        del hits[i]["metadata"]
    return [
        Text(
            text=hit["text"],
            metadata=ElementMetadata(
                url=source,
                page_number=hit["page_number"],
            ),
        ) for hit in hits
    ]

def load_statute_elements(collection_name: str):
    for chapter in chapter_names:
        statute_urls = load_statute_urls(chapter)
        for statute_url in statute_urls:
            yield vdb_source_documents(collection_name, statute_url)

def load_chapter_elements(collection_name: str):
    for chapter_pdf_url in chapter_pdf_urls:
        yield vdb_source_documents(collection_name, chapter_pdf_url)

#### 2.6 Reader - LLM 💬

>In this part, the LLM Reader reads the retrieved documents to formulate its answer.

OpenProBono offers various LLM Readers using models offered by OpenAI, HuggingFace, and more. We will create a simple LLM Reader connected to the OpenAI API for this notebook.

In [ ]:
RAG_PROMPT_TEMPLATE = """
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{context}

Question: {question}
"""

RAG_PROMPT_UNFORMATTED = (
    "Using the information contained in the context, "
    "give a comprehensive answer to the question. "
    "Respond only to the question asked, response should be concise and relevant to the question. "
    "Provide the number of the source document when relevant. "
    "If the answer cannot be deduced from the context, do not give an answer."
)

VANILLA_PROMPT = (
    "Give a comprehensive answer to the question. "
    "Respond only to the question asked, response should be concise and relevant to the question. "
    "If the answer cannot be deduced, do not give an answer."
    "\n\nQuestion: {question}"
)

In [ ]:
def answer_with_rag(
    collection_name: str,
    question: str,
    k: int = 7,
) -> tuple[str, list[dict]]:
    """Answer a question using RAG."""
    # Gather documents with retriever
    relevant_docs = milvusdb.query(collection_name, question, k)
    relevant_docs = [doc["entity"]["text"] for doc in relevant_docs["result"]]  # keep only the text

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {i!s}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = call_llm(llm_client, final_prompt, temperature=0)

    return answer, relevant_docs

### 3: Benchmarking the RAG System

>The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evlauation dataset.
>
>To this end, **we setup a judge agent**. ⚖️🤖
>
>Out of the different RAG evaluation metrics, we choose to focus only on faithfulness since it the best end-to-end metric of our system's performance.

>💡 *In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in Prometheus's prompt template: this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples.*
>
>💡 *Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement.*

In [ ]:
def run_rag_tests(
    eval_dataset: pd.DataFrame,
    collection_name: str,
    output_file: str,
    verbose: bool | None = True,
    test_settings: str = "",  # To document the test settings used
):
    """Run RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with Path(output_file).open() as f:
            outputs = json.load(f)
    except:
        outputs = []

    for _, example in tqdm(eval_dataset.iterrows(), total=len(eval_dataset)):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(collection_name, question)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": list(relevant_docs),
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with Path(output_file).open("w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_SYSTEM_MSG = "You are a fair evaluator language model."

EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

In [ ]:
evaluator_name = llm_name
eval_temperature = 0


def evaluate_answers(
    answer_path: str,
    evaluator_name: str,
) -> None:
    """Evaluate generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if Path(answer_path).is_file():  # load previous generations if they exist
        with Path(answer_path).open() as f:
            answers = json.load(f)

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_msg = {"role": "user", "content": eval_prompt}
        eval_result = call_llm(llm_client, EVALUATION_SYSTEM_MSG, evaluator_name, eval_temperature, [eval_msg])
        feedback, score = (item.strip() for item in eval_result.split("[RESULT]"))
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with Path(answer_path).open(mode="w") as f:
            json.dump(answers, f)

>🚀 Let's run the tests and evaluate answers!👇

In [ ]:
from datetime import UTC, datetime

if not Path("./output").exists():
    Path.mkdir("./output")

vdb_basename = "EmploymentEval_" + datetime.now(UTC).strftime("%Y%m%d")
idx = 1
for chunk_hardmax, chunk_softmax, overlap in [(2500, 1000, 250)]:
    for encoder in [
        EncoderParams(name=OpenAIModelEnum.embed_small, dim=768),
    ]:
        settings_name = (
            f"chunk-hardmax:{chunk_hardmax}"
            f"_chunk-softmax:{chunk_softmax}"
            f"_embeddings:{encoder.name}"
            f"_dim:{encoder.dim}"
            f"_reader-model:{llm_name}"
        )
        output_file_name = f"./output/rag_{settings_name}.json"

        print(f"Running evaluation for {settings_name}:")

        print("Loading knowledge base embeddings...")
        collection_name = f"{vdb_basename}_{idx}"
        idx += 1
        if pymilvus.utility.has_collection(collection_name):
            collection = pymilvus.Collection(collection_name)
        else:
            description = (
                f"Hardmax = {chunk_hardmax}, "
                f"Softmax = {chunk_softmax}, "
                f"Overlap = {overlap}, "
                f"Encoder = {encoder.name}."
            )
            collection = create_collection_function(
                collection_name,
                encoder,
                description,
            )
            populate_database(collection, chunk_hardmax, chunk_softmax, overlap, encoder)

        print("Running RAG...")
        run_rag_tests(
            eval_dataset=couples_df,
            collection_name=collection_name,
            output_file=output_file_name,
            verbose=True,
            test_settings=settings_name,
        )

        print("Running evaluation...")
        evaluate_answers(
            output_file_name,
            evaluator_name,
        )

Plot the average evaluation scores as accuracy percentage:

In [ ]:
import matplotlib.pyplot as plt

results_gpt = pd.read_json("output/employment_eval_gpt35.json")["eval_score_gpt-3.5-turbo-0125"]
results_hive7b = pd.read_json("output/employment_eval_hive.json")["eval_score_gpt-3.5-turbo-0125"]
results_hive70b = pd.read_json("output/employment_eval_hive70b.json")["eval_score_gpt-3.5-turbo-0125"]

# Define the x-axis labels and values (evaluation models)
models = ["gpt-3.5-turbo-0125", "hive llm chat 7B", "hive llm chat 70B"]

# Define the y-axis values (average scores)
y_values = [(results_gpt.mean() / 5) * 100, (results_hive7b.mean() / 5) * 100, (results_hive70b.mean() / 5) * 100]

# Create the bar graph
plt.bar(models, y_values, color=["lightgreen", "lightblue", "blue"])

# Add values above each bar
for i, value in enumerate(y_values):
    plt.text(i, value, f"{value:.2f}", ha="center")

# Set title and labels
plt.title("Evaluation Accuracy Comparison")
plt.xlabel("Model")
plt.ylabel("Average Evaluation Accuracy")

# Show the plot
plt.show()

# for storing chunks
def make_jsonl():
    with Path("data/ncstatutes.jsonl").open("w") as f:
        for pdf_fname in tqdm(os.listdir("data/pdfs/")):
            if not pdf_fname.endswith(".pdf"):
                continue
            elements = partition(filename="data/pdfs/" + pdf_fname)
            chunks = chunk_by_title(elements, max_characters=2500, combine_text_under_n_chars=500, new_after_n_chars=1000, overlap=250)
            for chunk in chunks:
                json.dump({"text":chunk.text, "metadata":{"source":chunk.metadata.filename, "page_number":chunk.metadata.page_number}}, f)
                f.write("\n")